In [ ]:
import numpy as np 
import pandas as pd
import itertools

In [ ]:
"""
DIMENSIONLESS WEIGHTED FERMAT OPTIMIZATION

This script calculates all possible bifurcation angles for
given stream velocities, widths, and depths. 

This script will call warnings from Pandas, depending on your version. 

R. Strickland, July 2025
__________________________________________________________

Dimensionless ratios:
u_star_i = u_i / u_0
h_star_i = h_i / h_0
w_star_i = w_i / w_0

"""
"""
---USER INPUTS---
"""
depth_ratio = 100 #Ice stream width to depth ratio (~10-100)

"""
Naming convention:
greenland = 10:1 width to depth ratio
antarctica = 100:1 width to depth ratio
lwb = low-weight branches
hwb = high-weight branches
"""
save = "yes" #string, "yes" or "no"
fname = "output/full_sweep_antarctica_hwb"


bw = 1.5 #branch weights

#Parent channel
u_0 = 1 #Dimensionless velocity 
w_0 = 1 #Dimensionless width
h_0 = w_0 / depth_ratio #Dimensionless depth

#Minimum and maximum allowable velocity ratios 
#Velocities that fall outside of this are deemed unphysical
#These data were not used in the paper
v_min = 0.1
v_max = 2.

#Minimum and maximum reasonable velocity ratios 
#Velocities that fall outside of this are deemed unlikely
#These data are presented in the paper
v_min_opt = 0.5
v_max_opt = 1.5

#Range of dimensionless widths for bifurcate branches
w_1 = np.linspace(0.1, 2, 25)
w_2 = np.linspace(0.1, 2, 25)

#Range of dimensionless depths for bifurcate branches

h_1 = np.linspace(0.5, 1.5, 25) / depth_ratio
h_2 = np.linspace(0.5, 1.5, 25) / depth_ratio

#Range of flux ratios in bifurcate branches
f = np.linspace(0.1, 0.9, 25)

#Scale the importance of energy dissipation through basal friction 
# in parent channel and bifurcate branches
B0 = [1]
B1 = [1]
B2 = [1]

#Scale the importance of energy dissipation through lateral shear deformation 
# in the margins in parent channel and bifurcate branches
m0 = 1
m1 = 1
m2 = 1

"""
---END USER INPUTS---
"""



#Dimensionless flux in parent channel
Q_0 = u_0 * w_0 * h_0

#Conservation of mass for dimensionless flux
Q_1 = f * Q_0
Q_2 = (1 - f) * Q_0

#Create a dataframe of all possible combinations
branches = list(itertools.product(w_1, h_1, w_2, h_2, Q_1, B0, B1, B2))

W_1 = np.array([combo[0] for combo in branches])
H_1 = np.array([combo[1] for combo in branches])
W_2 = np.array([combo[2] for combo in branches])
H_2 = np.array([combo[3] for combo in branches])
Qs_1 = np.array([combo[4] for combo in branches])
b0 = np.array([combo[5] for combo in branches])
b1 = np.array([combo[6] for combo in branches])
b2 = np.array([combo[7] for combo in branches])

#Dataframe with all possible combinations
df1 = pd.DataFrame({"width_1":W_1,
                   "height_1":H_1,
                   "flux_1":Qs_1,
                   "width_2":W_2,
                   "height_2":H_2,
                   "b_0":b0,
                   "b_1":b1,
                   "b_2":b2})

#Calcualte dimensionless cross-sectinoal areas and fluxes in 
# bifurcate branches
df1['area_1'] = df1['width_1'] * df1['height_1']
df1['area_2'] = df1['width_2'] * df1['height_2']
df1['velocity_1'] = df1['flux_1'] / df1['area_1']
df1['flux_2'] = Q_0 - df1['flux_1']
df1['velocity_2'] = df1['flux_2'] / df1['area_2']

#Find combinations that yield velocities within 
# a physically reasonable range
df1["possible"] =  np.where(((df1['velocity_1'] > v_max) | (df1['velocity_1'] < v_min)) | 
                            ((df1['velocity_2'] > v_max) | (df1['velocity_2'] < v_min)), 
                            'no', 
                            'yes')

#Find combinations that yield velocities within a likely range
df1["likely"] =  np.where(((df1['velocity_1'] > v_max_opt) | (df1['velocity_1'] < v_min_opt)) | 
                            ((df1['velocity_2'] > v_max_opt) | (df1['velocity_2'] < v_min_opt)), 
                            'no', 
                            'yes')

#Save all possible bifurcation combinations to a new dataframe 
tmp = df1[df1['possible']=='yes']

#Calculate energy dissipation in each branch
tmp['d0'] = (tmp['b_0'] * (Q_0)**2 / (w_0 * h_0**2) + 
             m0 * ((Q_0)**(4/3) * h_0**(2/3) / (w_0**(8/3))))

tmp['d1'] = bw * (tmp['b_1'] * (tmp['flux_1'])**2 / (tmp['height_1']**2 * tmp['width_1']) + 
             m1 * ((tmp['flux_1']**(4/3) * tmp['height_1'])**(2/3) / (tmp['width_1']**(8/3))))

tmp['d2'] = bw * (tmp['b_2'] * (tmp['flux_2'])**2 / (tmp['height_2']**2 * tmp['width_2']) + 
             m1 * ((tmp['flux_2']**(4/3) * tmp['height_2'])**(2/3) / (tmp['width_2']**(8/3))))

#Calcualte bifurcate branch angles
tmp['angle_1'] = np.degrees(np.arccos((tmp['d0']**2 + tmp['d1']**2 - tmp['d2']**2)/(2*tmp['d0']*tmp['d1'])))
tmp['angle_2'] = np.degrees(np.arccos((tmp['d0']**2 - tmp['d1']**2 + tmp['d2']**2)/(2*tmp['d0']*tmp['d2'])))
tmp['sum_angle'] = np.degrees(np.arccos((tmp['d0']**2 - tmp['d1']**2 - tmp['d2']**2)/(2*tmp['d1']*tmp['d2'])))

#Save only the combinations with valid branch angles
stable = tmp.dropna(axis = 0)

#Calculate imbalance numbers
stable['dissipation_ratio'] = stable['d0'] / (stable['d1'] + stable['d2'])
stable['branch_diss_imbalance'] = np.abs(stable['d1'] - stable['d2']) / (stable['d1'] + stable['d2'])
stable['branch_width_imbalance'] = np.abs(stable['width_1'] - stable['width_2']) / (stable['width_1'] + stable['width_2'])
stable['branch_depth_imbalance'] = np.abs(stable['height_1'] - stable['height_2']) / (stable['height_1'] + stable['height_2'])
stable['branch_velocity_imbalance'] = np.abs(stable['velocity_1']-stable['velocity_2'])/(stable['velocity_1']+stable['velocity_2'])
stable['branch_angle_imbalance'] = np.abs(stable['angle_1']-stable['angle_2'])/(stable['angle_1']+stable['angle_2'])
stable['branch_flux_imbalance'] = np.abs(stable['flux_1']-stable['flux_2'])/(stable['flux_1']+stable['flux_2'])
stable['dissipation_imbalance'] = np.abs(stable['d0'] - stable['d1'] - stable['d2']) / (stable['d0'] + stable['d1'] + stable['d2'])

if save == "yes":
    stable.to_csv(fname + ".csv")